# 0. what for
1. 对大数据集进行清洗
# 1. Preparations
## 1.1 全局设置

In [56]:
#导入pandas包
import pandas as pd
import numpy as np
from tqdm import tqdm

auctions_selected_path = r'E:\DATA\large_dta\auctions_tmp.csv'      # 暂存data，只包含了必要的cols

# output
item_trans = r'E:\DATA\large_dta\items.csv'
auctions_cleaned_path = r'E:\DATA\large_dta\auctions_cleaned.csv'   # retail修正的data


- read in

## 1.2 可以用的columns重新save data
1. 'k','y'分别表示bid inc和bid fee
2. adjvalue是amazon价格，worthupto是PA网站给出的零售价，
    - 这篇paper里用作
3. data中的columns包括:`['Unnamed: 0', 'auctionid', 'endtime', 'item', 'worthupto', 'adjvalue',
        'endbidlevel', 'endbidlevel_norm', 'endtimeperiod', 'shipping_swp',
        'shipping_am', 'beginnerauction', 'nailbiter', 'percentoff',
        'fixedprice', 'pennyauction', 'username', 'profit', 'profitperc',
        'at_only', 'us_only', 'amazon_match', 'type', 'worthupto_de',
        'currency', 'worthupto_com', 'exchangerate', 'cash', 'freebids',
        'consumer', 'y', 'k']`
4. data_2中的columns包括：`['Unnamed: 0', 'auctionid', 'item', 'adjvalue', 'nailbiter', 'type', 'k',
       'time', 'netvalue', 'adjvalue_ln', 'netvalue_ln', 'sunkcosts_agg',
       'sunkcosts_agg_ln', 'end', 'hour', 'week', 'day', 'dayofweek',
       'valuebin']`
5. 新保存的data后缀为`selected`，**实际上我们只需要data这一个auction-level的table就够了**

In [92]:

# name=pd.read_stata(r'E:\DATA\large_dta\auctions.dta')
# name.to_csv(r'E:\DATA\large_dta\auctions.csv', encoding="utf-8",header=True,index = False)
# print('\n')
# print('转换成功...')




转换成功...


- 保留必要的columns

In [11]:
col_keep = ['auctionid',
    'item',
    'pennyauction','fixedprice','type',
    'currency','exchangerate',
    'adjvalue','worthupto',
    'endtimeperiod','k','y']

data = pd.read_csv(r'E:\DATA\large_dta\auctions.csv')
data_tmp = data[col_keep].copy()

- 统一列名

In [12]:
data_tmp.rename(columns={'k':'bidincrement',
                    'y':"bidfee",
                    'item':'desc',
                    'auctionid': 'auction_id',
                    'fixedprice':'flg_fixedprice',
                    'worthupto':'retail',
                    },inplace=True)
data_tmp.columns

Index(['auction_id', 'desc', 'pennyauction', 'flg_fixedprice', 'type',
       'currency', 'exchangerate', 'adjvalue', 'retail', 'endtimeperiod',
       'bidincrement', 'bidfee'],
      dtype='object')

- 转换单位

In [13]:

data_tmp['bidincrement'] = data_tmp.bidincrement*0.01
data_tmp.to_csv(auctions_selected_path, encoding = 'utf-8', index=False, header=True)
# data_tmp.head()
del(data_tmp)
del(data)

## 1.3 ReRead
1. 重新读入数据，只包括需要的columns

In [42]:
data = pd.read_csv(auctions_selected_path, encoding = 'utf-8')
data.head()

,auction_id,desc,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,retail,endtimeperiod,bidincrement,bidfee
0,144359,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2918,8.822994,9.029682,2.0,0.15,0.1
1,154753,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2774,8.929026,8.929026,1.0,0.15,0.1
2,147974,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.3235,9.039505,9.251265,2.0,0.15,0.1
3,150174,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2944,8.840752,9.047855,1.0,0.15,0.1
4,144356,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,0,0,Consumer Goods,Euro,1.2993,8.874219,9.082107,1.0,0.15,0.1


# 2. 异常筛选
## 2.1 drop data where ['bidfee'] >= ['retail']
1. 异常数据，bid fee比retail都高。
    - 有几十条
2. 注意这里的retail的概念
    - 有大概60%的商品可以在amazon上找到，对于这些商品，作者用amazon售价替换了PA网站的建议零售价，构成了*adjusted value*
    - amazon的价格只有PA的79%
    - 另一个数据集的retail的概念是PA网站提供的价格，虽然也提到了amazon售价是偏低的。

In [43]:
idx_drop = data[(data['bidfee'])>=data['retail']].index
data.drop(idx_drop,axis = 0,inplace=True)
print("Drop 掉了 *{}* 行data，他们的bidfee >= retail".format(len(idx_drop)))

Drop 掉了 *0* 行data，他们的bidfee >= retail


## 2.2 desc的问题
- 有无空值: 无


In [44]:
data.desc.isnull().any(),data.desc.isna().any()

(False, False)

In [45]:
data.desc.nunique()

9363

In [46]:
# data_desc = data.item.unique()
# pd.DataFrame(data_desc).to_csv(r'E:\DATA\large_dta\desc.csv', encoding = 'utf-8',index=False)

# 3. 基本规则筛选
## 3.1 only keep pennyauction
- 'pennyauction'这一列是boolean value:
    - 注意pennyauction远远少于非pennyauction
- 'pennyauction'和'fixedprice'有可能同时为1吗:
    - 有的，26行，
    - 这些行应该去掉吗: no

In [47]:
idx_drop = data[data.pennyauction==0].index
data.drop(index=idx_drop,inplace=True,axis=0)
# auction_id_1.shape
print(f"去掉了*{len(idx_drop)}*个data records, 它们不是penny auction")
print(f"现在剩 *{data.shape[0]}* 场auction")

去掉了*155670*个data records, 它们不是penny auction
现在剩 *10709* 场auction


## 3.2 bid inc=0且不是fixed—price的不需要
1. 他们是fixed-price吗：Not all of them

In [48]:
# inc==0且是增加拍卖
idx_drop= data[(data.bidincrement == 0)&(data.flg_fixedprice == 0)].index
data.drop(index=idx_drop,inplace=True,axis=0)

print(f"去掉了*{len(idx_drop)}*个data records, 它们的bid inc=0且是增加拍卖")
print(f"现在剩 *{data.shape[0]}* 场auction")
data.reset_index(inplace=True,drop=True)

去掉了*7*个data records, 它们的bid inc=0且是增加拍卖
现在剩 *10702* 场auction


## 3.3 对于'desc'相同的，对'retail'进行统一
1. 一样的商品应该具有一样的价格: 取avg

In [50]:
# 计算新的retail
data_common_grouped = data.groupby(by=['desc'],as_index=False)
item_retail = pd.DataFrame(data_common_grouped.retail.mean())
item_retail['retail'] = np.rint(item_retail.retail)   # 四舍五入
item_retail.head()

,desc,retail
0,1 KG SILBERBARREN,559.0
1,10 GRAMM GOLDBARREN,325.0
2,10 GRAMM PLATINBARREN,366.0
3,10 MARK BAYERN GOLDMüNZE,268.0
4,1000 FREEBIDS-GUTSCHEIN!,671.0


In [51]:
# drop旧的retail
data.drop(columns='retail',inplace=True,axis=1)
# 合并新的
data_new_retail = pd.merge(data,item_retail,on=['desc'],how="left")
data_new_retail.head(10)

,auction_id,desc,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,endtimeperiod,bidincrement,bidfee,retail
0,138823,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,1,0,Consumer Goods,Euro,1.3978,9.546973,10.0,0.01,0.01,10.0
1,175928,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0
2,176047,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0
3,176073,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0
4,176010,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0
5,175995,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0
6,175925,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3500,13.500000,5.0,0.01,0.01,14.0
7,180618,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3963,13.962999,118.0,0.01,0.01,14.0
8,176043,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3448,13.448000,6.0,0.01,0.01,14.0
9,176006,20 BIDS GUTSCHEIN,1,0,Consumer Goods,Euro,1.3640,13.639999,42.0,0.01,0.01,14.0


## 3.4 翻译item

In [53]:
item = pd.DataFrame(data_new_retail.desc.unique())
print(f"一共有auction记录 *{data_new_retail.shape[0]}* 条，一共有 *{item.shape[0]} 条item*")
item.insert(1,"item_en","")
item.rename(columns={0:"desc"},inplace=True)
item.head()

一共有auction记录 *10702* 条，一共有 *764 条item*


,desc,item_en
0,KOSMETIK BüRSTEN SET 6 TEILIG AUS HOLZ,
1,20 BIDS GUTSCHEIN,
2,20 BIDS VOUCHER,
3,50 BIDS GUTSCHEIN,
4,50 FREEBIDS-GUTSCHEIN!,


In [54]:
import translators as ts
from_language, to_language = 'de', 'en'
for i in tqdm(range(item.shape[0])):
    text_de = item.iloc[i,0]
    text_en = ts.translate_text(text_de, from_language = from_language, to_language = to_language,if_ignore_empty_query=False, if_ignore_limit_of_length=False, limit_of_length=5000)
    item.iloc[i,1] = text_en

print("Done")

Using state Beijing server backend.
100%|██████████| 764/764 [10:45<00:00,  1.18it/s]

Done


In [57]:
# 暂存一下item翻译记录, 翻译太慢了
item.to_csv(item_trans,header=True,index=False,encoding="utf-8")

In [58]:
# 合并，重命名
data_en = pd.merge(data_new_retail,item,on="desc",how="left")
data_en.drop(columns=['desc'],inplace=True)
data_en.rename(columns={"item_en":"desc"},inplace=True)
data_en.head()

,auction_id,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,endtimeperiod,bidincrement,bidfee,retail,desc
0,138823,1,0,Consumer Goods,Euro,1.3978,9.546973,10.0,0.01,0.01,10.0,COSMETICS Brush SET 6 PIECES MADE OF WOOD
1,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
2,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
3,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER
4,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER


## 3.5 endtimeperiod一定是整数
1. 这一列就是N值本身

In [59]:
data_en['endtimeperiod'] = np.rint(data_en.endtimeperiod)   # 四舍五入
data_en.rename(columns={'endtimeperiod':'N'},inplace=True)

## 3.6 Save data


In [60]:
data_en.to_csv(auctions_cleaned_path,encoding="utf-8",index=False,header=True)